# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 13 2022 12:24PM,241774,10,8270103,"Citieffe, Inc.",Released
1,Jul 13 2022 12:22PM,241773,12,HH-35833,Hush Hush,Released
2,Jul 13 2022 12:22PM,241773,12,HH-35834,Hush Hush,Released
3,Jul 13 2022 12:22PM,241773,12,HH-35835,Hush Hush,Released
4,Jul 13 2022 12:22PM,241773,12,HH-35837,Hush Hush,Released
5,Jul 13 2022 12:22PM,241773,12,HH-35838,Hush Hush,Released
6,Jul 13 2022 12:15PM,241772,10,AERO713-01,Aerogen LTD.,Released
7,Jul 13 2022 12:02PM,241771,12,HH6350,Hush Hush,Released
8,Jul 13 2022 11:56AM,241770,10,SOIVA0000496,Ivaoes Animal Health,Released
9,Jul 13 2022 11:56AM,241770,10,SOIVA0000497,Ivaoes Animal Health,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,241770,Released,3
44,241771,Released,1
45,241772,Released,1
46,241773,Released,5
47,241774,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241770,NaN,3.0
241771,NaN,1.0
241772,NaN,1.0
241773,NaN,5.0
241774,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241604,15.0,7.0
241670,48.0,0.0
241672,20.0,17.0
241676,0.0,2.0
241679,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241604,15,7
241670,48,0
241672,20,17
241676,0,2
241679,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241604,15,7
1,241670,48,0
2,241672,20,17
3,241676,0,2
4,241679,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241604,15,7
1,241670,48,
2,241672,20,17
3,241676,,2
4,241679,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 13 2022 12:24PM,241774,10,"Citieffe, Inc."
1,Jul 13 2022 12:22PM,241773,12,Hush Hush
6,Jul 13 2022 12:15PM,241772,10,Aerogen LTD.
7,Jul 13 2022 12:02PM,241771,12,Hush Hush
8,Jul 13 2022 11:56AM,241770,10,Ivaoes Animal Health
11,Jul 13 2022 11:50AM,241769,10,"Methapharm, Inc."
12,Jul 13 2022 11:14AM,241764,10,Eywa Pharma Inc.
17,Jul 13 2022 11:10AM,241711,15,"Person & Covey, Inc."
46,Jul 13 2022 10:56AM,241712,15,"Carwin Pharmaceutical Associates, LLC"
52,Jul 13 2022 10:55AM,241759,16,Sartorius Stedim Filters Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 13 2022 12:24PM,241774,10,"Citieffe, Inc.",,1
1,Jul 13 2022 12:22PM,241773,12,Hush Hush,,5
2,Jul 13 2022 12:15PM,241772,10,Aerogen LTD.,,1
3,Jul 13 2022 12:02PM,241771,12,Hush Hush,,1
4,Jul 13 2022 11:56AM,241770,10,Ivaoes Animal Health,,3
5,Jul 13 2022 11:50AM,241769,10,"Methapharm, Inc.",1,
6,Jul 13 2022 11:14AM,241764,10,Eywa Pharma Inc.,,5
7,Jul 13 2022 11:10AM,241711,15,"Person & Covey, Inc.",8,21
8,Jul 13 2022 10:56AM,241712,15,"Carwin Pharmaceutical Associates, LLC",,6
9,Jul 13 2022 10:55AM,241759,16,Sartorius Stedim Filters Inc.,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 13 2022 12:24PM,241774,10,"Citieffe, Inc.",1,
1,Jul 13 2022 12:22PM,241773,12,Hush Hush,5,
2,Jul 13 2022 12:15PM,241772,10,Aerogen LTD.,1,
3,Jul 13 2022 12:02PM,241771,12,Hush Hush,1,
4,Jul 13 2022 11:56AM,241770,10,Ivaoes Animal Health,3,
5,Jul 13 2022 11:50AM,241769,10,"Methapharm, Inc.",,1
6,Jul 13 2022 11:14AM,241764,10,Eywa Pharma Inc.,5,
7,Jul 13 2022 11:10AM,241711,15,"Person & Covey, Inc.",21,8
8,Jul 13 2022 10:56AM,241712,15,"Carwin Pharmaceutical Associates, LLC",6,
9,Jul 13 2022 10:55AM,241759,16,Sartorius Stedim Filters Inc.,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 13 2022 12:24PM,241774,10,"Citieffe, Inc.",1,
1,Jul 13 2022 12:22PM,241773,12,Hush Hush,5,
2,Jul 13 2022 12:15PM,241772,10,Aerogen LTD.,1,
3,Jul 13 2022 12:02PM,241771,12,Hush Hush,1,
4,Jul 13 2022 11:56AM,241770,10,Ivaoes Animal Health,3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 13 2022 12:24PM,241774,10,"Citieffe, Inc.",1.0,NaN
1,Jul 13 2022 12:22PM,241773,12,Hush Hush,5.0,NaN
2,Jul 13 2022 12:15PM,241772,10,Aerogen LTD.,1.0,NaN
3,Jul 13 2022 12:02PM,241771,12,Hush Hush,1.0,NaN
4,Jul 13 2022 11:56AM,241770,10,Ivaoes Animal Health,3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 13 2022 12:24PM,241774,10,"Citieffe, Inc.",1.0,0.0
1,Jul 13 2022 12:22PM,241773,12,Hush Hush,5.0,0.0
2,Jul 13 2022 12:15PM,241772,10,Aerogen LTD.,1.0,0.0
3,Jul 13 2022 12:02PM,241771,12,Hush Hush,1.0,0.0
4,Jul 13 2022 11:56AM,241770,10,Ivaoes Animal Health,3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3626086,126.0,85.0
12,967035,15.0,20.0
15,1208446,53.0,50.0
16,1450444,13.0,13.0
19,483496,2.0,0.0
20,1208587,47.0,23.0
21,483411,2.0,0.0
22,483493,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3626086,126.0,85.0
1,12,967035,15.0,20.0
2,15,1208446,53.0,50.0
3,16,1450444,13.0,13.0
4,19,483496,2.0,0.0
5,20,1208587,47.0,23.0
6,21,483411,2.0,0.0
7,22,483493,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,126.0,85.0
1,12,15.0,20.0
2,15,53.0,50.0
3,16,13.0,13.0
4,19,2.0,0.0
5,20,47.0,23.0
6,21,2.0,0.0
7,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,126.0
1,12,Released,15.0
2,15,Released,53.0
3,16,Released,13.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Executing,85.0,20.0,50.0,13.0,0.0,23.0,0.0,0.0
Released,126.0,15.0,53.0,13.0,2.0,47.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Executing,85.0,20.0,50.0,13.0,0.0,23.0,0.0,0.0
1,Released,126.0,15.0,53.0,13.0,2.0,47.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Executing,85.0,20.0,50.0,13.0,0.0,23.0,0.0,0.0
1,Released,126.0,15.0,53.0,13.0,2.0,47.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()